# Keycloak OIDC Experiments

> Testing Keycloak OIDC flows

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import base64
import hashlib
import html
import json
import os
import re
import urllib.parse
import requests
import uuid
import prettyprinter
from prettyprinter import pprint



In [ ]:
# HTTP Response Formatter 
prettyprinter.install_extras(['requests'])
def print_response(response):
    print(f'STATUS CODE: {response.status_code}' + "\n")
    print(f'HEADERS: {prettyprinter.pformat(response.headers)}' + "\n")
    print(f'TEXT: {response.text}')
    

In [ ]:
provider     = "https://localhost:8037/realms/test"
client_id    = "account-console"
redirect_uri = "https://localhost:8037/realms/test/account/#/"

In [ ]:
code_verifier = base64.urlsafe_b64encode(os.urandom(40)).decode('utf-8')
code_verifier = re.sub('[^a-zA-Z0-9]+', '', code_verifier)
code_verifier, len(code_verifier)

('imRZNaZnh55wxckFw5fuv01N8Ldm0zdU4ZREFfFG8eSvm9PDKw', 50)

In [ ]:
code_challenge = hashlib.sha256(code_verifier.encode('utf-8')).digest()
code_challenge = base64.urlsafe_b64encode(code_challenge).decode('utf-8')
code_challenge = code_challenge.replace('=', '')
code_challenge, len(code_challenge)

('PDbXWodkxLRxllxCm6so8QgsW7FUeTg0Ihqd-MR5Ack', 43)

In [ ]:
state = uuid.uuid4() # Random UUID for state
state

UUID('b2a84dde-768e-45bc-b693-4d7db8ccbdf1')

In [ ]:
# Disable security warnings. Don't do this in production.
requests.packages.urllib3.disable_warnings()

In [ ]:
#|notest
resp = requests.get(
    url=provider + "/protocol/openid-connect/auth",
    params={
        "client_id": client_id,
        "redirect_uri": redirect_uri,
        "state": state,
        "response_mode": "fragment",
        "response_type": "code",
        "scope": "openid",
        ## @TODO Is 'nonce' needed?
        # "nonce": "228d2420-311a-4619-9963-3ea86378b78f",
        "code_challenge": code_challenge,
        "code_challenge_method": "S256",
    },
    verify=False, # \\Disables TLS cert verification. Don't do this in production.
    allow_redirects=False
)
print_response(resp)

STATUS CODE: 200

HEADERS: requests.structures.CaseInsensitiveDict({
    'Referrer-Policy': 'no-referrer',
    'X-Frame-Options': 'SAMEORIGIN',
    'Strict-Transport-Security': 'max-age=31536000; includeSubDomains',
    'X-Robots-Tag': 'none',
    'Cache-Control': 'no-store, must-revalidate, max-age=0',
    'X-Content-Type-Options': 'nosniff',
    'Content-Security-Policy': "frame-src 'self'; frame-ancestors 'self'; object-src 'none';",
    'Set-Cookie':
        'AUTH_SESSION_ID=62a52327-adf9-4c59-8e54-44565c0d813f.d42be559ddc7-117'
        '12; Version=1; Path=/realms/test/; SameSite=None; Secure; HttpOnly, '
        'AUTH_SESSION_ID_LEGACY=62a52327-adf9-4c59-8e54-44565c0d813f.d42be559d'
        'dc7-11712; Version=1; Path=/realms/test/; HttpOnly, '
        'KC_RESTART=eyJhbGciOiJIUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI1MzNiMm'
        'IxOS04NTliLTQwN2ItYjk2ZC1mZjVkOTM2NTZhNjAifQ.eyJjaWQiOiJhY2NvdW50LWNv'
        'bnNvbGUiLCJwdHkiOiJvcGVuaWQtY29ubmVjdCIsInJ1cmkiOiJodHRwczovL2xvY2Fsa'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()